GRAB FIRST 1000 ENTRIES OF ABSD

In [21]:
from Bio import SeqIO
from pathlib import Path

input_path = Path("absd.fasta") 
output_path = Path("absd_1000.fasta")

records = []
for i, record in enumerate(SeqIO.parse(input_path, "fasta")):
    if i >= 1000:
        break
    records.append(record)

for record in records:
    print(record + "\n")

ID: 7UAP|||7UAP_3|Chains
Name: 7UAP|||7UAP_3|Chains
Description: 7UAP|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;AbDb|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;CoV-AbDab-PDB|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;PDB|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;SACS|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;SAbDab|||7UAP_H_L_Homo sapiens_human_light;7UAP_H_L;IGLV4;PLAbDab|||7UAP_M_N_Homo sapiens_human_light;7UAP_M_N;IGLV4;PLAbDab|||7UAP_O_P_Homo sapiens_human_light;7UAP_O_P;IGLV4;PLAbDab|||7UAQ_3|Chain C[auth L]|C1520 Fab Light Chain|Homo sapiens (9606);7uaq;IGLV4;AbDb|||7UAQ_3|Chain C[auth L]|C1520 Fab Light Chain|Homo sapiens (9606);7uaq;IGLV4;CoV-AbDab-PDB|||7UAQ_3|Chain C[auth L]|C1520 Fab Lig

RUN RIOT ON FIRST 1000 ENTRIES OF ABSD, PREP IT FOR LINCLUST

In [22]:
from riot_na import create_riot_nt, Organism, Scheme
riot_nt = create_riot_nt(allowed_species=[Organism.HOMO_SAPIENS]) # default to human, adjust as need

results = []
for record in records:
    try:
        result = riot_nt.run_on_sequence(
            header=record.description,
            query_sequence=str(record.seq),
            scheme=Scheme.IMGT
        )
        results.append(result)
    except Exception as e:
        print(f"Error processing record {record.description}: {e}")

# Print results
for result in results:
    print(result.__dict__)

{'sequence_header': '7UAP|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;AbDb|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;CoV-AbDab-PDB|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;PDB|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;SACS|||7UAP_3|Chains E[auth L], G[auth N], I[auth P]|C1520 Fab Light Chain|Homo sapiens (9606);7uap;IGLV4;SAbDab|||7UAP_H_L_Homo sapiens_human_light;7UAP_H_L;IGLV4;PLAbDab|||7UAP_M_N_Homo sapiens_human_light;7UAP_M_N;IGLV4;PLAbDab|||7UAP_O_P_Homo sapiens_human_light;7UAP_O_P;IGLV4;PLAbDab|||7UAQ_3|Chain C[auth L]|C1520 Fab Light Chain|Homo sapiens (9606);7uaq;IGLV4;AbDb|||7UAQ_3|Chain C[auth L]|C1520 Fab Light Chain|Homo sapiens (9606);7uaq;IGLV4;CoV-AbDab-PDB|||7UAQ_3|Chain C[auth L]|C1520 Fab Light Chain|Homo sapiens (9606);7uaq;IGLV4;PDB|

TRANSPORT AND INTERPRET WITH LINCLUST / MMSEQS2

In [23]:
import subprocess
from pathlib import Path

# Create directory for results
results_dir = Path("linclust_results")
results_dir.mkdir(exist_ok=True)

# Define paths for MMseqs2 files
seqdb_path = results_dir / "seqDB"
clu_path = results_dir / "seqDB_clu"
tmp_path = results_dir / "tmp"
clusters_path = results_dir / "clusters.tsv"
input_fasta = results_dir / "processed_sequences.fasta"

# Save processed sequences to FASTA for MMseqs2
with open(input_fasta, "w") as f:
    for result in results:
        if result.sequence_aa:  # Use amino acid sequence if available
            f.write(f">{result.sequence_header}\n{result.sequence_aa}\n")
        else:
            f.write(f">{result.sequence_header}\n{result.sequence}\n")


SUBPROCESSES GENERATION FROM FASTA

In [24]:
# Create MMseqs2 database from FASTA
subprocess.run([
    "mmseqs", 
    "createdb", 
    str(input_fasta), 
    str(seqdb_path)
])

# Perform clustering
subprocess.run([
    "mmseqs",
    "cluster",
    str(seqdb_path),
    str(clu_path),
    str(tmp_path)
])

# Create TSV output of clustering results
subprocess.run([
    "mmseqs",
    "createtsv",
    str(seqdb_path),
    str(seqdb_path),
    str(clu_path),
    str(clusters_path)
])

createdb linclust_results/processed_sequences.fasta linclust_results/seqDB 

MMseqs Version:       	17-b804f
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[
Time for merging to seqDB_h: 0h 0m 0s 8ms
Time for merging to seqDB: 0h 0m 0s 5ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 43ms
Create directory linclust_results/tmp
cluster linclust_results/seqDB linclust_results/seqDB_clu linclust_results/tmp 

MMseqs Version:                     	17-b804f
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max

CompletedProcess(args=['mmseqs', 'createtsv', 'linclust_results/seqDB', 'linclust_results/seqDB', 'linclust_results/seqDB_clu', 'linclust_results/clusters.tsv'], returncode=0)

CLUSTER INTERPRETATIONS AND STATISTICS AS RECOMMENDED BY MMSEQS2

In [25]:
clusters = {}
with open(clusters_path) as f:
    for line in f:
        rep_seq, member_seq = line.strip().split("\t")
        if rep_seq not in clusters:
            clusters[rep_seq] = []
        clusters[rep_seq].append(member_seq)

# Print cluster statistics
print(f"Number of clusters: {len(clusters)}")
for rep_seq, members in clusters.items():
    print(f"Cluster {rep_seq} has {len(members)} members")

Number of clusters: 3
Cluster 11793|||11793_englumafusp has 1 members
Cluster PRJNA670581|SRR12875354|human|GGAGCAATCAGATAAG|IGHV5|||PRJNA670581|SRR12875354|human|GGAGCAATCAGATAAG|IGHV5_51*03,IGHD6_19*01,IGHJ4*02,IGHA1|IGLV4_69*01,IGLJ3*02,IGLC2|ARGAVAVDS|QTWGSGTQV_heavy;SRR12875354;IGHV5;PairedNGS|||SRR12875354-1404_GGAGCAATCAGATAAG-1_contig_2 has 500 members
Cluster ERR4082303-1221|||ERR4082303-1221_CGGAGTCCAATAGCGG-1_contig_1 has 499 members
